In [1]:
import os
import sys
import shutil
import subprocess
from pathlib import Path
from multiprocessing import Pool

import pandas as pd

In [2]:
sys.path.append('../src')

from utils import run_cmd

In [3]:
def run_porechop(inpath, outfile, threads=2):
    run_cmd(f'porechop -i {inpath} -t {threads} -o {outfile}', False, False)

In [4]:
dirpath = Path('/media/NAS/NanoporeOutput/20210617_Sal/guppy_barcode')
outpath = Path('/media/NGS/Nanopore_1/20210617_Sal/fastq')

In [5]:
with Pool(8) as p:
    try:
        for fp in dirpath.iterdir():
            if fp.is_dir() and fp.name != 'unclassified':
                outfile = outpath/(fp.name + '.fastq')
                p.apply_async(func=run_porechop, args=(fp, outfile, 8))
            elif fp.suffix == '.fastq':
                outfile = outpath/(fp.stem + '.fastq')
                p.apply_async(func=run_porechop, args=(fp, outfile, 8))
        p.close()
        p.join()
    except KeyboardInterrupt:
        p.terminate()

In [22]:
run_porechop(
    '/media/NGS/Data_Analysis/20201130_rapid_diagnostic_with_nanopore/mixed_samples/experiment-1/fastq/0_to_100.fastq',
    '/media/NGS/Data_Analysis/20201130_rapid_diagnostic_with_nanopore/mixed_samples/experiment-1/porechop_output/0_to_100.fastq',
    8)

In [3]:
dirpath = Path('/media/NGS/Nanopore_1/20200925/fastq/nanopore')
outpath = Path('/media/NGS/Nanopore_1/20200925/fastq/nanopore_trim/')

cmds = []
for filepath in dirpath.iterdir():
    outfile = outpath/filepath.name
    cmd = f'nanoq -f {filepath} -o {outfile} -q 10'
    cmds.append(cmd)

In [4]:
with Pool(12) as p:
    p.map(run_cmd, cmds)
    p.close()
    p.join()